In [1]:
import logging

# set up logging into the file
logging.basicConfig(
    filename="logs.txt",
    filemode="w",
    format="%(asctime)s %(levelname)s %(message)s",
    level=logging.INFO,
)

In [2]:
import yaml
import pandas as pd
from gpt_fhir.llmExtractor import LLMExtractor
from gpt_fhir.fhirTools import FHIRTools
from gpt_fhir.fhir import FHIR
from gpt_fhir.annotator import Annotator

# load the config file
with open("../config.yaml", "r") as f:
    config = yaml.safe_load(f)

# set up annotator
annotator = Annotator()

# set up FHIR client
fhir = FHIR(config, annotator)

# set up FHIR tools
fhir_tools = FHIRTools(config, fhir)

# set up llm chat
llm_extractor = LLMExtractor(config, fhir_tools)

In [9]:
# empty the resources
fhir.empty_resources()

# test llm extractor
text = "patient has a history of diabetes"
llm_extractor.extract(text)

# print extracted FHIR resources
print(fhir.get_resources())

[{'clinicalStatus': {'text': 'resolved'}, 'code': {'coding': [{'code': '127012008', 'display': 'Lipoatrophic diabetes', 'system': 'http://snomed.info/sct'}], 'text': 'Diabetes'}, 'subject': {'reference': 'Patient/1'}, 'resourceType': 'Condition'}]


In [4]:
# run FHIR extractor on all patients
data = pd.read_csv("../data/fhir_notes.csv")

# process each row
def extract_fhir(d):

    # empty FHIR resources
    fhir.empty_resources()

    # log entry
    logging.info(f"NOTE: {d['note']}")

    # run llm extractor
    llm_extractor.extract(d["note"])

    # return the created procedure
    return fhir.get_resources()


# apply the function to each row
data["extracted"] = data.apply(extract_fhir, axis=1)

# save the data with extracted FHIR resources
data.to_csv("../data/fhir_notes_extracted.csv", index=False)